In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [ ]:
import os
from glob import glob
from copy import copy
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
random_seed = 0
np.random.seed(random_seed)

In [ ]:
!ls -lh ../input

In [ ]:
all_data = pd.read_csv('../input/train/train.csv')
list(all_data.columns)

In [ ]:
pet_data = all_data.set_index('PetID')
image_data = []

for image in glob('../input/train_images/*.jpg'):
    basename = os.path.basename(image)
    pet_id, _ = basename.rsplit('-')
    pet_row = pet_data.loc[pet_id].to_dict()
    pet_row['ImageFilename'] = image
    pet_row['ImageBasename'] = basename
    image_data.append(pet_row)
    
image_data = pd.DataFrame(image_data)

image_data.head(2)

In [ ]:
image_data['AdoptionSpeed']= image_data['AdoptionSpeed'].astype(str)

In [ ]:
image_data = image_data.loc[image_data['Type'] == 1]
#image_data_cat = image_data.loc[image_data['Type'] == 2]

In [ ]:
len(image_data), len(pet_data)

In [ ]:
y = image_data['AdoptionSpeed']
test_size = 0.2
validation_size = 0.2

# Split the training data off from leftover (i.e. validation and testing)
# train_test_split(*arrays, **options)
# random_state is the seed used by the random number generator
# data is split in a stratified fashion, using this as the class labels
X_train, X_leftover, y_train, y_leftover = train_test_split(
    image_data, y, test_size=test_size, random_state=random_seed,
    stratify=y.values # stratify to ensure equal distribution of classes
)

# Determine how much the leftover section should be split to test
test_split = test_size / (test_size + validation_size)

X_validate, X_test, y_validate, y_test = train_test_split(
    X_leftover, y_leftover, test_size=test_split, random_state=random_seed,
    stratify=y_leftover.values # stratify to ensure equal distribution of classes
)

X_train.shape, X_validate.shape, X_test.shape

In [ ]:
#X_train['AdoptionSpeed'].hist(bins=3)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image_data.dtypes

In [ ]:
#from imblearn.keras import BalancedBatchGenerator
#from imblearn.under_sampling import NearMiss
#training_generator = BalancedBatchGenerator.flow_from_dataframe(
#     X, y, sampler=NearMiss(), batch_size=10, random_state=42)

In [ ]:
BATCH_SIZE = 30

In [ ]:
# Generate batches of tensor image data with real-time data augmentation. 
# The data will be looped over (in batches).



train_generator = train_datagen.flow_from_dataframe(
    X_train.reset_index(), # Need to reset index due to bug in flow_from_dataframe
    directory='../input/train_images/',
    x_col='ImageBasename',
    y_col='AdoptionSpeed',
    target_size=(150, 150),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
)

val_generator = val_datagen.flow_from_dataframe(
    X_validate.reset_index(), # Need to reset index due to bug in flow_from_dataframe
    directory='../input/train_images/',
    x_col='ImageBasename',
    y_col='AdoptionSpeed',
    target_size=(150, 150),
    color_mode='rgb',
    class_mode='categorical',
    batch_size= BATCH_SIZE,
)

In [ ]:
from collections import Counter
counter = Counter(train_generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

In [ ]:
#X_train

In [ ]:
#train_datagen = ImageDataGenerator(
#    rescale=1./255)

In [ ]:

#train_generator = train_datagen.flow_from_dataframe(
#    X_train.reset_index(), # Need to reset index due to bug in flow_from_dataframe
#    directory='../input/train_images/',
#    x_col='ImageBasename',
#    y_col='AdoptionSpeed',
#    target_size=(150, 150),
#    color_mode='rgb',
#    class_mode='categorical',
#    batch_size=20)

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
print ('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# import regularizer
from keras.regularizers import l1
# instantiate regularizer
reg = l1(0.001)

# example of l1 norm on activity from a cnn layer
from keras.layers import Conv2D
from keras.regularizers import l1

In [ ]:
from keras import regularizers


In [ ]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), 
                 activity_regularizer=l1(0.001))(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(5, activation='softmax')(x)

# Dropout is a a technique used to tackle Overfitting . 
# The Dropout method in keras.layers module takes in a 
# float between 0 and 1, which is the fraction of the 
# neurons to drop. 

# Configure and compile the model
model = Model(pre_trained_model.input, x)


In [ ]:
from tensorflow.keras.optimizers import SGD

unfreeze = False

# Unfreeze all models after "mixed6"
for layer in pre_trained_model.layers:
  if unfreeze:
    layer.trainable = True
  if layer.name == 'mixed6':
    unfreeze = True

# As an optimizer, here we will use SGD 
# with a very low learning rate (0.00001)
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(
                  lr=0.0001, 
                  momentum=0.9),
              metrics=['acc'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=len(X_train)//BATCH_SIZE,
      epochs=100,
      validation_data=val_generator,
      validation_steps=len(X_validate)//BATCH_SIZE,
      workers=4,
      class_weight=class_weights,
      verbose=1)

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')